In [5]:
import numpy as np
import pandas as pd
from scipy import interpolate

In [ ]:
    # return (1.-0.5*x*w1/w)*(1.-0.5*x*w1/w) - 0.25*w1*w1*(0.25 + 1./w) + 0.5*w2

In [6]:
def svi_raw(x, params):
    a, b, rho, m, sigma = params
    v = a+b*(rho*(x-m)+np.sqrt(np.square(x-m)+np.square(sigma)))
    return v

def svi1(x, t, params):
    a, b, rho, m, sigma = params
    return b*rho*t+b*t*(x-m)/np.sqrt(np.square(x-m)+np.square(sigma))

def svi2(x, t, params):
    a, b, rho, m, sigma = params
    return b*t*np.square(sigma)/(np.square(x-m)+np.square(sigma))**1.5

def total_variance(x, t, params):
    return svi_raw(x,params)*t

In [2]:
params = {.16:[.03, .125, -1., .074, .05],
          .26:[.032, .094, -1., .093, .041], 
          .33:[.028, .105, -1, .096, .072], 
          .58:[.026, .08, -1., .127, .098]}
tt = list(params.keys())

In [44]:
def g(x, t, params):
    w = svi_raw(x,params)*t
    w1 = svi1(x, t, params)
    w2 = svi2(x, t, params)
    wt = svi_raw(x, params)
    g = (1.-.5*x*w1/w)*(1.-.5*x*w1/w)-.25*w1*w1*(.25+1./w)+.5*w2
    return g

def local_variance(x, t, tt, params):
    wt = np.array([svi_raw(x, params[i]) for i in tt])
    funct = interpolate.UnivariateSpline(tt, wt, s=0, k=1)
    gt = np.array([g(x, i, params[i]) for i in tt])
    funcg = interpolate.UnivariateSpline(tt, gt, s=0, k=2)
    return funct(t)/funcg(t)

In [42]:
g(.2, .2, params[.16])

1.0697259611939036

In [45]:
T = np.linspace(0, .1, 100)
X = np.linspace(-.2, .2, 100)
lv = pd.DataFrame([[local_variance(i, j, tt, params) for i in X] for j in T], columns=X, index=T)
lv.min().min()

0.01595643140912676

In [58]:
def localvol_MonteCarlo(s0, K, rf, q, t, tt, params, M=100, N=10000, flag=1):
    
    s = np.ones((N,))*s0
    dt = t/M
    tau = t
    x = np.log(K/(s*np.exp((rf-q)*tau)))
    ss = []
    for i in range(M):
        dw = np.random.randn(N, )*np.sqrt(dt)
        sigma = np.array([np.sqrt(local_variance(j, tau, tt, params)) for j in x])
        # sigma = np.array([local_variance(j, tau, tt, params) for j in x])
        s = s*np.exp((rf-q-.5*np.square(sigma))*dt+sigma*dw)
        tau -= dt
        x = np.log(K/(s*np.exp((rf-q)*tau)))
        pay_off = s-K
        pay_off = pay_off[pay_off > 0]
        V = pay_off.sum()/(N*np.exp((rf-q)*t))
        
    return V

In [60]:
localvol_MonteCarlo(100, 105, .03, .0, .5, tt, params)

5.256979806419607

In [51]:
def svi_surface(x, t, tt, params):
    w = np.array([svi_raw(x, params[i]) for i in tt])
    func = interpolate.UnivariateSpline(tt, w, s=0, k=1)
    return np.sqrt(func(t))

In [52]:
rf, q, t = .03, 0., .5
F = np.log(100/(105*np.exp((rf-q)*t)))
svi_surface(F, t, tt, params)

0.2448364143530775

In [53]:
from scipy.stats import norm

def BlackScholesCore(CallPutFlag,DF,F,X,T,v):
    ## DF: discount factor
    ## F: Forward
    ## X: strike
    vsqrt=v*np.sqrt(T)
    d1 = (np.log(F/X)+(vsqrt*vsqrt/2.))/vsqrt
    d2 = d1-vsqrt
    if CallPutFlag:
        return DF*(F*norm.cdf(d1)-X*norm.cdf(d2))
    else:
        return DF*(X*norm.cdf(-d2)-F*norm.cdf(-d1))
    
##  Black-Scholes Pricing Function
def BlackScholes(CallPutFlag,S,X,T,r,d,v):
    ## r, d: continuous interest rate and dividend
    return BlackScholesCore(CallPutFlag,np.exp(-r*T),np.exp((r-d)*T)*S,X,T,v)



In [63]:
c1 = BlackScholes(1, 100, 105, t, rf, q, svi_surface(F, t, tt, params))
c2 = localvol_MonteCarlo(100, 105, .03, .0, .5, tt, params)
print("bsm:", c1)
print("LocalVol_MonteCarlo:", c2)

bsm: 5.431125795216952
LocalVol_MonteCarlo: 4.721396971555975
